In [1]:
import torch
import gym

from collections import defaultdict
env=gym.make('Blackjack-v0')

![image.png](attachment:image.png)

# STEP 1: Define Run Episode Function

In [2]:
def run_episode(env,Q,epsilon,n_action):
    #1.Initialize the first state and create lists for the rewards, actions and states during the simulation (As always)
    state=env.reset()
    rewards=[]
    actions=[]
    states=[]
    is_done=False
    #2.Initilize a loop that breaks when the player get a score>=21
    while not is_done:
    #3.Creates probabilities for action 0 and action 1. Epsilon goes up to 1 if we do not explore. If epsilon=0 we take all actions at random
        probs=torch.ones(n_action)*epsilon/n_action
    #4.We extract the best action out of the Q function. If Q function does not have this state it autmatically generates it with'state' as a key and creates a tensor (111,1111232)  as an item.  
        best_action=torch.argmax(Q[state]).item()
    #5.It gives the best action according to the Q function  a probability the largest weight.  
        probs[best_action]+=1-epsilon
    #6.Samples a number between 0,1 with probabilities discribed before
        action=torch.multinomial(probs,1).item()
    #7.Put all actions into a list
        actions.append(action)
        states.append(state)
    #8.Now we finally take a step
        state,reward,is_done,info=env.step(action)
        rewards.append(reward)  
    #9.Break the loop if the game is over
        if is_done:
            break
    return states, actions, rewards

# STEP 2: MC Control with Epsilon Greedy

In [3]:
def mc_control_epsilon_greedy(env,gamma,n_episode,epsilon):
    #1.We define the action space
    n_action=env.action_space.n
    #2.This vector is going to capture the agreggate Gsum 
    G_sum=defaultdict(float)
    #3.The N vector is going to capture the number of times I visit a particular state
    N=defaultdict(int)
    #4.Q Functions captures the payoffs of taking actions across every state fx. Q(0,a)=[Q(0,1),Q(0,0)]
    Q=defaultdict(lambda:torch.empty(n_action))
    #5.We start running multiple episodes
    for episode in range(n_episode):
        states_t, actions_t, rewards_t = run_episode(env,Q,epsilon,n_action)
        return_t=0
    #6.G Calculates the payoffs and states of an episode
        G={}
    #7.We loop starting from the end state,at the end of this loop we will have dictionary that captures state_action and returns visitied in the episode    
        for state_t, action_t, reward_t in zip(states_t[::-1],actions_t[::-1], rewards_t[::-1]):
            return_t=reward_t+gamma*return_t
            G[(state_t,action_t)]=return_t
    #8.'state_action' es el 'key' of the G dictionary and 'return_t' is the 'item'       
        for state_action , return_t in G.items(): 
            state, action=state_action
    #9.Si mis cartas son menores o iguales a 21        
            if state[0]<=21:
                G_sum[state_action]+= return_t
                N[state_action] +=1
    #10.El promedio vara cada estado accion queda guardado en su respectiva position en la funcion Q            
                Q[state][action]=G_sum[state_action]
    #11. When everything is done we create a policy with the best values of the Q functions for every state
    policy={}
    for state,actions in Q.items():
        policy[state]=torch.argmax(actions).item()
    return Q, policy
    

In [ ]:
gamma=1
n_episode=500000
epsilon=0.1

optimal_q, optimal_policy=mc_control_epsilon_greedy(env,gamma,n_episode,epsilon)

In [ ]:
optimal_policy

# Checking wheter it really works!


In [ ]:
n_episode=100000
n_win_optimal=0
n_lose_optimal=0
for _ in range(n_episode)